In [1]:
import java.io.File
abstract class Problem<T>(private val filename: String) {
    abstract val testResult: T
    protected var isTesting: Boolean = true
        private set
    fun run() {
        val testOutput = File("testData/$filename").useLines { solve(it.toList()) }
        if (testOutput != testResult) {
            throw IllegalStateException("Test output doesn't match expected result \n Actual: $testOutput \n Expected: $testResult")
        }
        println("Test results: Expected=$testResult Actual=$testOutput")

        isTesting = false
        val actualOutput = File("data/$filename").useLines { solve(it.toList()) }
        println("Solution: $actualOutput")
    }
    abstract fun solve(lines: List<String>): T
}

## Day 5: Fresh Ingredients
### Part 1
As the forklifts break through the wall, the Elves are delighted to discover that there was a cafeteria on the other side after all.

You can hear a commotion coming from the kitchen. "At this rate, we won't have any time left to put the wreaths up in the dining hall!" Resolute in your quest, you investigate.

"If only we hadn't switched to the new inventory management system right before Christmas!" another Elf exclaims. You ask what's going on.

The Elves in the kitchen explain the situation: because of their complicated new inventory management system, they can't figure out which of their ingredients are fresh and which are spoiled. When you ask how it works, they give you a copy of their database (your puzzle input).

The database operates on ingredient IDs. It consists of a list of fresh ingredient ID ranges, a blank line, and a list of available ingredient IDs. For example:
```
3-5
10-14
16-20
12-18

1
5
8
11
17
32
```
The fresh ID ranges are inclusive: the range `3-5` means that ingredient IDs `3`, `4`, and `5` are all fresh. The ranges can also overlap; an ingredient ID is fresh if it is in any range.

The Elves are trying to determine which of the available ingredient IDs are fresh. In this example, this is done as follows:

Ingredient ID `1` is spoiled because it does not fall into any range.
Ingredient ID `5` is fresh because it falls into range `3-5`.
Ingredient ID `8` is spoiled.
Ingredient ID `11` is fresh because it falls into range `10-14`.
Ingredient ID `17` is fresh because it falls into range `16-20` as well as range `12-18`.
Ingredient ID `32` is spoiled.
So, in this example, `3` of the available ingredient IDs are fresh.

Process the database file from the new inventory management system. How many of the available ingredient IDs are fresh?

In [2]:
class DayFivePartOne(override val testResult: Int = 3) : Problem<Int>("day5.txt") {
    override fun solve(lines: List<String>): Int {
        val lineList = lines.toList()
        val okayIds = lineList
            .takeWhile { it.isNotBlank() }
            .map { it.substringBefore("-").toLong() to it.substringAfter("-").toLong() }
        val ingredients = lineList.dropWhile { it.contains("-") || it.isBlank() }.map { it.toLong() }
        val freshIds = mutableSetOf<Long>()
        okayIds.forEach { (start, end) ->
            ingredients.forEach { ingredient ->
                if (ingredient in start..end) {
                    freshIds.add(ingredient)
                }
            }
        }
        return freshIds.size
    }
}

DayFivePartOne().run()

Test results: Expected=3 Actual=3
Solution: 613


### Part 2
The Elves start bringing their spoiled inventory to the trash chute at the back of the kitchen.

So that they can stop bugging you when they get new inventory, the Elves would like to know all of the IDs that the fresh ingredient ID ranges consider to be fresh. An ingredient ID is still considered fresh if it is in any range.

Now, the second section of the database (the available ingredient IDs) is irrelevant. Here are the fresh ingredient ID ranges from the above example:
```
3-5
10-14
16-20
12-18
```
The ingredient IDs that these ranges consider to be fresh are `3`, `4`, `5`, `10`, `11`, `12`, `13`, `14`, `15`, `16`, `17`, `18`, `19`, and `20`. So, in this example, the fresh ingredient ID ranges consider a total of `14` ingredient IDs to be fresh.

Process the database file again. How many ingredient IDs are considered to be fresh according to the fresh ingredient ID ranges?

In [3]:
class DayFivePartTwo(override val testResult: Long = 14) : Problem<Long>("day5.txt") {
    override fun solve(lines: List<String>): Long {
        val okayIds = lines
            .takeWhile { it.isNotBlank() }
            .map { it.substringBefore("-").toLong() to it.substringAfter("-").toLong() }
            .sortedBy { it.first }
        val condensedFreshIds: MutableSet<Pair<Long, Long>> = mutableSetOf()
        okayIds.forEach { freshRange ->
            val toUpdate = condensedFreshIds.firstOrNull { condensedRange -> freshRange.overlaps(condensedRange) }
            if (toUpdate == null) {
                condensedFreshIds.add(freshRange)
            } else {
                condensedFreshIds.remove(toUpdate)
                condensedFreshIds.add(min(freshRange.first, toUpdate.first) to max(freshRange.second, toUpdate.second))
            }
        }
        return condensedFreshIds.sumOf { it.second - it.first + 1 }
    }

    private fun Pair<Long, Long>.overlaps(other: Pair<Long, Long>): Boolean =
        this.first in other.first..other.second || this.second in other.first..other.second
}

DayFivePartTwo().run()

Test results: Expected=14 Actual=14
Solution: 336495597913098
